In [1]:
from models.transformers import torch_Transformer
from src.utils import model_utils

import torch



vocab_size = 10000
d_model = 512  
n_head = 8
en_layers = 6
d_ff = 2048
max_seq_length = 600


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


tr_tf_en = torch_Transformer.TRTF_encoder(vocab_size, d_model, 
    n_head, en_layers, d_ff, max_seq_length)

tr_tf_en = tr_tf_en.to(device)
tr_tf_en = tr_tf_en.bfloat16()



print(sum(p.numel() for p in tr_tf_en.parameters())/1e9, 'B parameters in tr_tf_en')




src = torch.randint(0, vocab_size, (10, 32))
src_mask = model_utils.square_subsequent_mask(10)


src = src.to(device)

src_mask = src_mask.to(device)
src_mask = src_mask.bfloat16()


output = tr_tf_en.forward(src,src_mask)
print("TRTF_en output shape",output.shape)



d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
0.024341504 M parameters in tr_tf_en
TRTF_en output shape torch.Size([10, 32, 512])


In [2]:
import json
from transformers import BertTokenizer
from src.utils import data_utils

with open('./data/RL_env/Pettingzoo_mpe/simple_adversary.json','r') as file:
    data = json.load(file)

data = data_utils.json2Tpath(data)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_data = tokenizer(data, padding=True, truncation= True, max_length=4096, return_tensors="pt")




In [3]:
from models.transformers import torch_Transformer
from src.utils import model_utils

import torch



vocab_size = 10000
d_model = 512  
n_head = 8
en_layers = 6
d_ff = 2048
max_seq_length = 4096
out_dim = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


ptf_trtf_en = torch_Transformer.paddingT_FTRTF_encoder(vocab_size, d_model, 
    n_head, en_layers, d_ff, out_dim, max_seq_length)

ptf_trtf_en = ptf_trtf_en.to(device)
ptf_trtf_en = ptf_trtf_en.bfloat16()



print(sum(p.numel() for p in ptf_trtf_en.parameters())/1e9, 'B parameters in tr_tf_en')




with torch.no_grad():
    input_ids = tokenized_data['input_ids'].to(device)
    attention_mask = tokenized_data['attention_mask'].to(device)
    outputs = ptf_trtf_en(input_ids, attention_mask)

print(outputs.shape)


cuda


TypeError: super(type, obj): obj must be an instance or subtype of type